# Cross-Asset Volatility Spillover: Data Integrity & Sanity Checks

## Learning Objectives
The goal of this project is not to find profitable alpha.
The objectives are:
- Clean, validate, and align intraday financial data while avoiding leakage.
- Understand cross-asset relationships, especially volatility spillovers.
- Apply a disciplined research workflow from data → features → signals → returns.

This notebook will only focus on the first objective. 

# Cleaning and aligning financial data 

In [68]:
import pandas as pd

In [54]:

df = pd.read_csv(
    "/Users/nathr/Downloads/HISTDATA_COM_ASCII_EURUSD_M1202511/eurusdnov.csv",
    sep=";",
    header=None,
    names=["datetime", "open", "high", "low", "close", "volume"]
)

# Split datetime
df["datetime"] = pd.to_datetime(
    df["datetime"],
    format="%Y%m%d %H%M%S"
)

df = df.set_index("datetime")

print(df.head())


                        open     high      low    close  volume
datetime                                                       
2025-11-02 17:00:00  1.15294  1.15324  1.15294  1.15324       0
2025-11-02 17:01:00  1.15324  1.15329  1.15289  1.15328       0
2025-11-02 17:02:00  1.15328  1.15328  1.15328  1.15328       0
2025-11-02 17:03:00  1.15295  1.15328  1.15295  1.15328       0
2025-11-02 17:04:00  1.15295  1.15295  1.15295  1.15295       0


### Time Step Consistency Check
We examine time differences between consecutive bars to verify that the data is sampled at the intended frequency.

In [58]:
dt = df.index.to_series().diff()
dt.value_counts().head(10)


datetime
0 days 00:01:00    28974
0 days 00:02:00       92
0 days 00:03:00       11
0 days 00:04:00        3
2 days 00:01:00        2
2 days 00:03:00        1
2 days 00:05:00        1
0 days 00:05:00        1
Name: count, dtype: int64

Most observations occur at exactly 1-minute intervals, confirming correct resampling.  
A small number of larger gaps (2–5 minutes) likely reflect illiquid periods.  
Multi-day gaps correspond to weekends when FX markets are closed.

No forward-filling is applied, as filling missing intervals would introduce artificial price dynamics and potential data leakage.


### Large Gap Verification
We explicitly inspect time gaps greater than five minutes.

In [60]:
dt[dt > pd.Timedelta("5min")].head(20)


datetime
2025-11-09 17:00:00   2 days 00:01:00
2025-11-16 17:02:00   2 days 00:03:00
2025-11-23 17:04:00   2 days 00:05:00
2025-11-30 17:00:00   2 days 00:01:00
Name: datetime, dtype: timedelta64[ns]

Observed gaps of approximately two days align with expected FX weekend closures.  
This confirms that the dataset respects real-world trading hours rather than artificially enforcing continuity.

### Price Level Sanity Check
We summarize the OHLC price distributions to ensure values are economically realistic.

In [62]:
bad_rows = df[
    (df["high"] < df[["open","close"]].max(axis=1)) |
    (df["low"]  > df[["open","close"]].min(axis=1)) |
    (df["low"]  > df["high"])
]

len(bad_rows)


0

No logical inconsistencies found. 

In [64]:
df[["open","high","low","close"]].describe()


,open,high,low,close
count,29086.000000,29086.000000,29086.000000,29086.000000
mean,1.155907,1.155965,1.155849,1.155907
std,0.004040,0.004042,0.004039,0.004040
min,1.146940,1.147100,1.146870,1.146950
25%,1.152530,1.152590,1.152470,1.152530
50%,1.156460,1.156530,1.156400,1.156460
75%,1.159070,1.159120,1.159020,1.159070
max,1.165480,1.165590,1.165430,1.165490


EUR/USD prices remain within expected ranges, with no abnormal spikes or near-zero values.  
The observed volatility is consistent with intraday FX behavior.

This suggests prices were parsed and scaled correctly.

### Missing Data Check
We verify the presence of missing values across OHLC and volume columns.


In [66]:
df.isna().sum()


open      0
high      0
low       0
close     0
volume    0
dtype: int64

No missing values are detected, indicating that resampling and aggregation produced a complete time series without gaps inside trading hours.

## Data Cleaning steps are repeated with the second dataset: EURGBP. 

In [70]:
df2 = pd.read_csv(
    "/Users/nathr/Downloads/HISTDATA_COM_ASCII_EURGBP_M1202511/eurgbpnov.csv",
    sep=";",
    header=None,
    names=["datetime", "open", "high", "low", "close", "volume"]
)

# Split datetime
df2["datetime"] = pd.to_datetime(
    df2["datetime"],
    format="%Y%m%d %H%M%S"
)

df2 = df2.set_index("datetime")

print(df2.head())


                        open     high      low    close  volume
datetime                                                       
2025-11-02 17:04:00  0.87737  0.87737  0.87737  0.87737       0
2025-11-02 17:05:00  0.87738  0.87772  0.87711  0.87772       0
2025-11-02 17:06:00  0.87773  0.87773  0.87740  0.87773       0
2025-11-02 17:07:00  0.87772  0.87773  0.87741  0.87741       0
2025-11-02 17:08:00  0.87740  0.87771  0.87740  0.87771       0


In [72]:
dt2 = df2.index.to_series().diff()
dt2.value_counts().head(10)

datetime
0 days 00:01:00    28894
0 days 00:02:00      104
0 days 00:03:00       13
0 days 00:04:00        8
0 days 00:05:00        5
2 days 00:05:00        3
2 days 00:01:00        1
Name: count, dtype: int64

In [74]:
dt2[dt2 > pd.Timedelta("5min")].head(20)

datetime
2025-11-09 17:04:00   2 days 00:05:00
2025-11-16 17:04:00   2 days 00:05:00
2025-11-23 17:04:00   2 days 00:05:00
2025-11-30 17:00:00   2 days 00:01:00
Name: datetime, dtype: timedelta64[ns]

In [99]:
bad_rows = df2[
    (df2["high"] < df2[["open","close"]].max(axis=1)) |
    (df2["low"]  > df2[["open","close"]].min(axis=1)) |
    (df2["low"]  > df2["high"])
]

len(bad_rows)


0

In [101]:
df2[["open","high","low","close"]].describe()


,open,high,low,close
count,29029.000000,29029.000000,29029.000000,29029.000000
mean,0.879850,0.879896,0.879805,0.879850
std,0.002389,0.002391,0.002388,0.002389
min,0.874600,0.874630,0.874550,0.874590
25%,0.878100,0.878160,0.878050,0.878100
50%,0.880250,0.880300,0.880190,0.880250
75%,0.881620,0.881670,0.881570,0.881620
max,0.886500,0.886510,0.886380,0.886480


In [105]:
df2.isna().sum()


open      0
high      0
low       0
close     0
volume    0
dtype: int64

## Both datasets are combined 

In [115]:
combined = df.merge(df2, how='inner', left_index=True, right_index=True, suffixes=('_eurusd','_eurgbp'))
combined.head()

,open_eurusd,high_eurusd,low_eurusd,close_eurusd,volume_eurusd,open_eurgbp,high_eurgbp,low_eurgbp,close_eurgbp,volume_eurgbp
datetime,,,,,,,,,,
2025-11-02 17:04:00,1.15295,1.15295,1.15295,1.15295,0,0.87737,0.87737,0.87737,0.87737,0
2025-11-02 17:05:00,1.15306,1.15329,1.15295,1.15319,0,0.87738,0.87772,0.87711,0.87772,0
2025-11-02 17:06:00,1.15300,1.15331,1.15300,1.15302,0,0.87773,0.87773,0.87740,0.87773,0
2025-11-02 17:07:00,1.15303,1.15309,1.15303,1.15305,0,0.87772,0.87773,0.87741,0.87741,0
2025-11-02 17:08:00,1.15307,1.15309,1.15306,1.15309,0,0.87740,0.87771,0.87740,0.87771,0


In [117]:
dt = combined.index.to_series().diff()
print(dt.value_counts().head(10))


datetime
0 days 00:01:00    28717
0 days 00:02:00      157
0 days 00:03:00       27
0 days 00:04:00       10
0 days 00:05:00        8
2 days 00:05:00        3
0 days 00:06:00        1
2 days 00:01:00        1
Name: count, dtype: int64


In [133]:
# EUR/USD
bad_eurusd = combined[
    (combined["high_eurusd"] < combined[["open_eurusd", "close_eurusd"]].max(axis=1)) |
    (combined["low_eurusd"]  > combined[["open_eurusd", "close_eurusd"]].min(axis=1)) |
    (combined["low_eurusd"]  > combined["high_eurusd"])
]
print("EUR/USD bad rows:", len(bad_eurusd))

# EUR/GBP
bad_eurgbp = combined[
    (combined["high_eurgbp"] < combined[["open_eurgbp", "close_eurgbp"]].max(axis=1)) |
    (combined["low_eurgbp"]  > combined[["open_eurgbp", "close_eurgbp"]].min(axis=1)) |
    (combined["low_eurgbp"]  > combined["high_eurgbp"])
]
print("EUR/GBP bad rows:", len(bad_eurgbp))


EUR/USD bad rows: 0
EUR/GBP bad rows: 0


In [137]:
# EUR/USD
combined[["open_eurusd", "high_eurusd", "low_eurusd", "close_eurusd"]].describe()


,open_eurusd,high_eurusd,low_eurusd,close_eurusd
count,28925.000000,28925.000000,28925.000000,28925.000000
mean,1.155909,1.155968,1.155851,1.155909
std,0.004040,0.004041,0.004039,0.004040
min,1.146940,1.147100,1.146870,1.146950
25%,1.152530,1.152590,1.152470,1.152530
50%,1.156470,1.156530,1.156400,1.156470
75%,1.159070,1.159110,1.159020,1.159070
max,1.165480,1.165590,1.165430,1.165490


In [135]:

# EUR/GBP
combined[["open_eurgbp", "high_eurgbp", "low_eurgbp", "close_eurgbp"]].describe()


,open_eurgbp,high_eurgbp,low_eurgbp,close_eurgbp
count,28925.000000,28925.000000,28925.000000,28925.000000
mean,0.879856,0.879902,0.879811,0.879857
std,0.002387,0.002388,0.002385,0.002387
min,0.874600,0.874630,0.874550,0.874590
25%,0.878120,0.878170,0.878070,0.878120
50%,0.880250,0.880310,0.880200,0.880250
75%,0.881620,0.881670,0.881580,0.881620
max,0.886500,0.886510,0.886380,0.886480


In [122]:
combined.isna().sum()


open_eurusd      0
high_eurusd      0
low_eurusd       0
close_eurusd     0
volume_eurusd    0
open_eurgbp      0
high_eurgbp      0
low_eurgbp       0
close_eurgbp     0
volume_eurgbp    0
dtype: int64


**Key points for combining datasets:**  

- Only overlapping timestamps should be used for cross-asset analysis to avoid misalignment or lookahead bias.  
- Document gaps in the data; they can indicate missing market activity or data provider filtering.  
- Resampling or forward-filling can be used carefully if required, but **never fill gaps with future data**, as that introduces leakage.  
